In [1]:
#Makes data paths
import os
import json
path = os.path.abspath('{}/../'.format('transfer_learning_experiment.ipynb') )
data_path = '{}/mmimdb/dataset/'.format(path)
import sys
import pandas as pd
import glob
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/transfer/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


## Example of a plot summary with human-applied genre targets

In [2]:
with open('{}0399877.json'.format(data_path)) as json_data:
    data = json.load(json_data)
    print(data['genres'])
    print(data['title'])
    print(data['plot'])

['Documentary', 'Comedy', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi']
What the #$*! Do We (K)now!?
['"WHAT THE #$*! DO WE KNOW?!" is a radical departure from convention. It demands a freedom of view and greatness of thought so far unknown, indeed, not even dreamed of since Copernicus. It\'s a documentary. It\'s a story. It\'s mind-blowing special effects. This film plunges you into a world where quantum uncertainty is demonstrated - where neurological processes, and perceptual shifts are engaged and lived by its protagonist - where everything is alive, and reality is changed by every thought.', "Amanda, a divorced photographer, finds herself in a fantastic Alice-in-Wonderland experience when her daily, uninspired life literally begins to unravel, revealing the cellular, molecular and even quantum worlds which lie beneath. Guided by a Greek Chorus of leading scientists and mystics, she finds that if reality itself is not questionable, her notion of it certainly is. Stunning special effects

In [3]:
texts_df = pd.read_csv("text_labels.csv")
texts_df.head()

,tags,text
0,Crime Drama Thriller,Le cercle rouge On the eve of his release afte...
1,Comedy Drama Music Romance,Adam & Steve Teenage Goth couple Adam and Rhon...
2,Sci-Fi,"Stereo Sometime in the future, the Canadian Ac..."
3,Comedy,Pledge This! At popular South Beach University...
4,Comedy Fantasy Romance,Angel on My Shoulder Gangster Eddie Kagel is k...


In [4]:
texts_df.describe()

,tags,text
count,25959,25959
unique,2652,25951
top,Drama,Riget The Kingdom is the most technologically ...
freq,2681,2


In [5]:
genres = []
genre_count = {}

sys.path.insert(0, data_path)
for file in glob.glob('{}*.json'.format(data_path)):
    data = json.load(open(file))
    [genres.append(genre) for genre in data['genres']]

counts = Counter(genres)

KeyboardInterrupt: 

In [ ]:
counts_df = pd.DataFrame.from_dict(list(dict(counts).items()))
counts_named_df = counts_df.rename(columns={0:'Genre',1: '# Movies'})
counts_ordered_df = counts_named_df.sort_values(by='# Movies')

In [ ]:
counts_ordered_df

In [7]:
#To match Arevelo's experiment, don't remove 'Film-Noir', 'Short', 'Sport'
genres_to_remove = ['Film-Noir',
                    'Short',
                    'Sport'
                    'Reality-TV', 
                    'Talk-Show', 
                    'Adult', 
                    'News']

In [ ]:
texts_lower_genres_rm_df = texts_df.dropna()

In [ ]:
texts_lower_genres_rm_df.loc[:,'tags'] = \
    texts_lower_genres_rm_df.tags.apply(
        lambda tags: ' '.join([tag for tag in tags.split() \
                               if tag not in genres_to_remove]))

In [ ]:
texts_no_dups_df = texts_lower_genres_rm_df.drop_duplicates(subset='text')

In [ ]:
texts_tag_counts = texts_no_dups_df
texts_tag_counts.loc[:, 'tag_count'] = texts_tag_counts.tags.apply(
                                        lambda x: len(x.split(' ')))

In [ ]:
texts_tag_word_counts = texts_tag_counts
texts_tag_word_counts.loc[:, 'plot_word_count'] = texts_tag_counts.text.apply(
                                        lambda x: len(x.split(' ')))

In [ ]:
texts_tag_word_counts.to_csv("{}text_mod_labels.csv".format(path),index=False)

In [6]:
#Language Model
import torch
import fastai
from fastai.text import * 

In [8]:
#USE GPU - Device 0 
torch.cuda.set_device(0)

RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at torch/csrc/cuda/Module.cpp:32

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'text_mod_labels.csv', text_cols = 'text')
data_lm.save()
data_lm = TextLMDataBunch.load(make_data_path) #TextLMDataBunch expects to find data_lm saved to a temp directory

In [ ]:
# Language model creation with WT103
language_model = language_model_learner(data_lm, pretrained_model=URLs.WT103_1, drop_mult=0.5)

In [ ]:
language_model.fit_one_cycle(1, 1e-2)

In [ ]:
# Unfreeze all layers and fit to MM-IMDB data (data_lm)
language_model.unfreeze()
language_model.fit_one_cycle(1, 1e-3)

In [ ]:
language_model.predict("This movie really", n_words=15)

In [ ]:
#Save the language model
language_model.save_encoder('lm_encoder')

In [ ]:
#Text Classifier
#Split data into train and validate sets
from sklearn.model_selection import train_test_split
texts_df = pd.read_csv(path+'text_mod_labels.csv')
train, valid = train_test_split(texts_df, test_size = 0.02, random_state = 0)

In [ ]:
# Create classifier model data
data_multilabel = TextClasDataBunch.from_df(path, 
                                      train_df = train,
                                      valid_df = valid,
                                      text_cols = 'text', 
                                      label_cols ='tags',
                                      label_delim=' ', 
                                      vocab=data_lm.train_ds.vocab, 
                                      bs=32)
data_multilabel.save()
data_multilabel = TextClasDataBunch.load(path, bs=32)

In [ ]:
text_classifier = text_classifier_learner(data_multilabel, drop_mult=0.5, metrics = [accuracy_thresh, fbeta])
print("learn classifier defined. model summary:")
print(text_classifier.model)
text_classifier.load_encoder('{}/models/lm_encoder'.format(make_data_path))

In [ ]:
text_classifier.fit_one_cycle(1, 1e-2)

In [ ]:
text_classifier.freeze_to(-2)
text_classifier.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [ ]:
text_classifier.unfreeze()
text_classifier.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [ ]:
text_classifier.predict('"WHAT THE #$*! DO WE KNOW?!" is a radical departure from convention. It demands a freedom of view and greatness of thought so far unknown, indeed, not even dreamed of since Copernicus. It\'s a documentary. It\'s a story. It\'s mind-blowing special effects. This film plunges you into a world where quantum uncertainty is demonstrated - where neurological processes, and perceptual shifts are engaged and lived by its protagonist - where everything is alive, and reality is changed by every thought.', "Amanda, a divorced photographer, finds herself in a fantastic Alice-in-Wonderland experience when her daily, uninspired life literally begins to unravel, revealing the cellular, molecular and even quantum worlds which lie beneath. Guided by a Greek Chorus of leading scientists and mystics, she finds that if reality itself is not questionable, her notion of it certainly is. Stunning special effects plunge you into a world where quantum uncertainty is demonstrated - where Amanda's neurological processes, and perceptual shifts are engaged and lived - where everything is alive, and reality is changed by every thought. This film gives voice to the modern day radical souls of science, making them the true heroes of our day as they conquer and map the greatest uncharted territory yet - man's consciousness itself.", 'It is part documentary, part story, and part elaborate and inspiring visual effects and animations. The protagonist, Amanda, played by Marlee Matlin, finds herself in a fantastic Alice in Wonderland experience when her daily, uninspired life literally begins to unravel, revealing the uncertain world of the quantum field hidden behind what we consider to be our normal, waking reality.')